In [1]:
import torch
import torch.nn as nn

In [2]:
BATCH_SIZE=4
SEQUENCE_LENGTH=5
INPUT_SIZE=2
HIDDEN_SIZE=3
NUM_LAYERS=1

In [3]:
lstm = nn.LSTM(input_size=INPUT_SIZE, hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS)

In [23]:
# input to hidden weights and biases
w_ih = lstm.weight_ih_l0
b_ih = lstm.bias_ih_l0

w_ii = w_ih[0*HIDDEN_SIZE:1*HIDDEN_SIZE]
w_if = w_ih[1*HIDDEN_SIZE:2*HIDDEN_SIZE]
w_ig = w_ih[2*HIDDEN_SIZE:3*HIDDEN_SIZE]
w_io = w_ih[3*HIDDEN_SIZE:]

b_ii = b_ih[0*HIDDEN_SIZE:1*HIDDEN_SIZE]
b_if = b_ih[1*HIDDEN_SIZE:2*HIDDEN_SIZE]
b_ig = b_ih[2*HIDDEN_SIZE:3*HIDDEN_SIZE]
b_io = b_ih[3*HIDDEN_SIZE:]

# hidden to hidden weights and biases
w_hh = lstm.weight_hh_l0
b_hh = lstm.bias_hh_l0

w_hi = w_hh[0*HIDDEN_SIZE:1*HIDDEN_SIZE]
w_hf = w_hh[1*HIDDEN_SIZE:2*HIDDEN_SIZE]
w_hg = w_hh[2*HIDDEN_SIZE:3*HIDDEN_SIZE]
w_ho = w_hh[3*HIDDEN_SIZE:]

b_hi = b_hh[0*HIDDEN_SIZE:1*HIDDEN_SIZE]
b_hf = b_hh[1*HIDDEN_SIZE:2*HIDDEN_SIZE]
b_hg = b_hh[2*HIDDEN_SIZE:3*HIDDEN_SIZE]
b_ho = b_hh[3*HIDDEN_SIZE:]

In [26]:
# create inputs to the LSTM
sequence = torch.randn(SEQUENCE_LENGTH, BATCH_SIZE, INPUT_SIZE)
h_0 = torch.zeros(NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE)
c_0 = torch.zeros(NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE)

In [50]:
output, (h_n, c_n) = lstm(sequence, (h_0, c_0))

In [51]:
def manual_lstm():
    hidden = h_0.clone()
    cell = c_0.clone()
    output = torch.zeros(SEQUENCE_LENGTH, BATCH_SIZE, HIDDEN_SIZE)
    with torch.inference_mode():
        for idx, seq in enumerate(sequence):
            f = torch.sigmoid(seq @ w_if.T + b_if + hidden[0] @ w_hf.T + b_hf)
            i = torch.sigmoid(seq @ w_ii.T + b_ii + hidden[0] @ w_hi.T + b_hi)
            o = torch.sigmoid(seq @ w_io.T + b_io + hidden[0] @ w_ho.T + b_ho)
            g = torch.tanh(seq @ w_ig.T + b_ig + hidden[0] @ w_hg.T + b_hg)
            
            cell[0] = f * cell[0] + i * g
            hidden[0] = o * torch.tanh(cell[0])
            output[idx] = hidden[0]
    return output, (hidden, cell)